In [24]:
from time import sleep
from selenium import webdriver
import csv

In [21]:
# Make sure you have the most recent Chrome Driver:
# https://sites.google.com/a/chromium.org/chromedriver/downloads

# Set path to Chrome Driver
chrome = '/Users/lucio/desktop/sos-scraper/chromedriver'

# Setting Chrome options
opt = webdriver.ChromeOptions()

# Chrome extension that disables images for faster load time
opt.add_extension("block-images.crx")

# Open Chrome browser
browser = webdriver.Chrome(executable_path = chrome, chrome_options=opt)

# Set URL
url = 'http://kepler.sos.ca.gov/'

In [ ]:
# Grabs data from csv file to run search 
data = open('data.csv', 'rb')
reader = csv.reader(data)
new_rows_list = []

count = 0

for row in reader:

      browser.get(url)
      html = browser.page_source
        
      if "ct100_content_placeholder_body_BusinessSearch1_RadioButtonList_SearchType_2" in html:
          browser.find_element_by_id("ct100_content_placeholder_body_BusinessSearch1_RadioButtonList_SearchType_2").click()
      else:
          browser.find_element_by_id("ctl00_content_placeholder_body_BusinessSearch1_RadioButtonList_SearchType_2").click()
    
      browser.find_element_by_css_selector('input[type=\"text\"]').send_keys('C'+row[1])  
      browser.find_element_by_css_selector('input[type=\"submit\"]').click()

      html = browser.page_source    
    
      if "valid entity number" in html:
          date = "null"
          status = "null"
          name = "null"
          agent = "null"
          location = "null"

      elif "Record not found" in html:
          date = "rnf"
          status = "rnf"
          name = "rnf"
          agent = "rnf"
          location = "rnf"
        
      else:
          date = browser.find_element_by_xpath('//td[2]').text
          status = browser.find_element_by_xpath('//td[3]').text
          name = browser.find_element_by_xpath('//td[4]/a').text
          agent = browser.find_element_by_xpath('//td[5]').text
          
          if 'ctl00_content_placeholder_body_SearchResults1_GridView_SearchResults_Corp_ctl02_ctl00' in html:
              browser.find_element_by_id('ctl00_content_placeholder_body_SearchResults1_GridView_SearchResults_Corp_ctl02_ctl00').click()
          else:
              browser.find_element_by_id('ctl00_content_placeholder_body_SearchResults1_GridView_SearchResults_Corp_ctl02_ctl00').click()
          location = browser.find_element_by_xpath('//tr[7]/td[2]').text
          
      new_row = [row[0], date, status, name, agent, location]
      new_rows_list.append(new_row)
        
      count += 1
#       print count
    
      if count % 5 == 0:
          sleep(5)

data.close()

In [23]:
# Do the writing
results = open('results.csv', 'wb')
writer = csv.writer(results)
writer.writerows(new_rows_list)
results.close()